# 앙상블 (mmdetection model + ultralytics YOLOv8) 평가

## 필요 라이브러리 한번에 다운로드

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
!pip install -e ../mmdetection

In [ ]:
!mim install "mmengine>=0.7.0"
!mim install "mmcv>=2.0.0rc4"

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../mmdetection')

## 데이터 준비

In [ ]:
# 앙상블 데이터 세팅
# import os

# # data 폴더 경로
# data_dir = 'data'
# output_dir = 'clean_rating_data'

# # output 디렉토리 생성
# os.makedirs(output_dir, exist_ok=True)

# # data 폴더 내의 모든 하위 폴더에 대해 반복
# for label_folder in os.listdir(data_dir):
#     label_folder_path = os.path.join(data_dir, label_folder)

#     # 폴더인 경우에만 처리
#     if os.path.isdir(label_folder_path):

#         # 폴더 내의 각 파일에 대해 반복
#         for filename in os.listdir(label_folder_path):
#             file_path = os.path.join(label_folder_path, filename)

#             # 파일이 실제로 파일이고 이미지 파일인 경우
#             if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
#                 # 폴더 이름을 라벨로 사용하여 파일 이름 변경
#                 new_filename = f'{label_folder}_{filename}'
#                 new_file_path = os.path.join(output_dir, new_filename)
#                 # 파일 이동
#                 os.rename(file_path, new_file_path)

In [ ]:
import os

def get_image_paths(data_dir):
    image_list = []

    # data 폴더 내의 모든 파일에 대해 반복
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)

        # 파일이 실제로 파일이고 이미지 파일인 경우
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_list.append(file_path)

    return image_list

In [ ]:
data_dir = './clean_rating_data'
images = get_image_paths(data_dir)

# 15장만 테스트
images = images[:15]

## ensemble 시각화

In [ ]:
import cv2
class_labels = ['bag', 'bottle', 'cafe-cup', 'can', 'dirty', 'keys', 'seat-dirty', 'socar-tissue', 'stain', 'trash', 'valuable']

# 각 클래스에 대한 색상 지정
class_colors = {
    'bag': (255, 0, 0),       # 빨간색
    'bottle': (0, 255, 0),    # 초록색
    'cafe-cup': (0, 0, 255),  # 파란색
    'can': (255, 255, 0),     # 노란색
    'dirty': (255, 0, 255),   # 자홍색
    'keys': (0, 255, 255),    # 청록색
    'seat-dirty': (255, 165, 0),      # 오렌지색
    'socar-tissue': (128, 0, 128),    # 보라색
    'stain': (128, 128, 0),    # 올리브색
    'trash': (0, 128, 128),    # 틸색
    'valuable': (255, 192, 203)  # 분홍색
}

def draw_detection_results(image, detection_results, img_width, img_height, normalize=False):
    # 복제하여 원본 이미지 변경 방지
    image_with_boxes = image.copy()

    for label, score, bbox in zip(detection_results['labels'], detection_results['scores'], detection_results['bboxes']):        
        if normalize:
            x1_norm, y1_norm, x2_norm, y2_norm = bbox

            # 정규화된 좌표를 이미지 크기에 맞게 변환
            x1 = int(x1_norm * img_width)
            y1 = int(y1_norm * img_height)
            x2 = int(x2_norm * img_width)
            y2 = int(y2_norm * img_height)
        else:
            x1, y1, x2, y2 = map(int, bbox)
        
        # 클래스 이름 가져오기
        class_name = class_labels[label]

        # 클래스에 대응하는 색상 가져오기
        color = class_colors.get(class_name, (0, 0, 0))  # 기본은 검은색

        # 상자 그리기
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)

        # 텍스트 추가 (클래스명과 신뢰도)
        label_text = f"Class: {class_name}, Score: {score:.2f}"
        cv2.putText(image_with_boxes, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image_with_boxes

In [ ]:
import matplotlib.pyplot as plt
import os
import shutil
from inference.clean_rating import determine_clean_class

clean_rate = ['pending', 'clean', 'normal', 'dirty']
result_directory = './visualization_result'

shutil.rmtree(result_directory, ignore_errors=True)
os.makedirs(result_directory, exist_ok=True)

In [ ]:
def save_visualization_result(images, mmdetection_predictions, yolov8_predictions, wbf_predictions):
    for idx, image in enumerate(images):
        file_name = os.path.basename(image)
        parts = file_name.split('_')
        label_class = parts[0]
        file_name = '_'.join(parts[1:])

        img = cv2.imread(image)

        final_width = img.shape[1]
        final_height = img.shape[0]

        mmdetection_img = cv2.resize(img, (1333, 800))
        yolov8_img = cv2.resize(img, (640, 640))
        final_img = cv2.resize(img, (final_width, final_height))

        mmdetection_image_with_boxes = draw_detection_results(mmdetection_img, mmdetection_predictions[idx], 1333, 800)
        yolov8_image_with_boxes = draw_detection_results(yolov8_img, yolov8_predictions[idx], 640, 640)
        final_image_with_boxes = draw_detection_results(final_img, wbf_predictions[idx], final_width, final_height, True)

        y_pred = determine_clean_class(wbf_predictions[idx])

        fig, axes = plt.subplots(3, 1, figsize=(15, 25))
        
        predicted_class = clean_rate[y_pred]

        is_correct = label_class == predicted_class
        title_color = 'green' if is_correct else 'red'
        result_title = f'{is_correct}, label : {label_class}, predict : {predicted_class}'

        plt.suptitle(result_title, fontsize=16, color=title_color)

        axes[0].imshow(cv2.cvtColor(mmdetection_image_with_boxes , cv2.COLOR_BGR2RGB))
        axes[0].set_title('dino Results')
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(yolov8_image_with_boxes, cv2.COLOR_BGR2RGB))
        axes[1].set_title('Yolov8 Results')
        axes[1].axis('off')

        axes[2].imshow(cv2.cvtColor(final_image_with_boxes, cv2.COLOR_BGR2RGB))
        axes[2].set_title('ensemble Results')
        axes[2].axis('off')

        result_filename = f'{is_correct}_ensemble_detection_{file_name}.png'
        result_path = os.path.join(result_directory, result_filename)

        fig.patch.set_facecolor('white')
        plt.savefig(result_path, facecolor='white')

        plt.close()

In [ ]:
# mmdetection
config_file = '../mmdetection/configs/dino/dino-4scale_r50_8xb2-60e_coco_all_v2.py'
checkpoint_file = './model/weights/mmdetection_dino_weights.pth'

In [ ]:
# yolov8
yolov8_checkpoint_file = "./model/weights/yolov8_weights.pt"

In [ ]:
from inference.predict_list import mmdetection_list_predict
from inference.predict_list import yolov8_list_predict
from inference.predict_list import ensemble_list_predict

for idx in range(0, len(images), 10):
    end_idx = min(idx + 10, len(images))
    mmdetection_predictions = mmdetection_list_predict(images[idx:end_idx] , config_file, checkpoint_file)
    yolov8_predictions = yolov8_list_predict(images[idx:end_idx] , yolov8_checkpoint_file)
    wbf_predictions = ensemble_list_predict(mmdetection_predictions, yolov8_predictions) 
    save_visualization_result(images[idx:end_idx], mmdetection_predictions, yolov8_predictions, wbf_predictions)
    
    print(f"{end_idx}/{len(images)} 시각화 완료...")